**Nivea Hayane - 500028**
Link do dataset: https://www.kaggle.com/datasets/mubeenshehzadi/the-state-of-student-minds

In [ ]:
from google.colab import drive
import pandas as pd
from IPython.display import display
import os
import pdfkit

In [ ]:
arquivo = '/content/drive/MyDrive/CD/Student Mental health.csv'
df = pd.read_csv(arquivo)

In [ ]:
# Necessário instalar wkhtmltopdf no Colab antes
!apt-get install wkhtmltopdf
!pip install pdfkit

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
wkhtmltopdf is already the newest version (0.12.6-2).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [ ]:
# Remove Timestamp pq n vai ser usado
if 'Timestamp' in df.columns:
    df = df.drop(columns=['Timestamp'])

df = df.convert_dtypes()

# Funções aux para forçar o tipo das colunas e ajustar os valores;
def exemplos(coluna, n=5):
    valores = coluna.dropna().unique()
    if len(valores) > n:
        valores = valores[:n]
    return ", ".join(map(str, valores))

def tipo_simplificado(serie, nome_coluna):
    nome = nome_coluna.lower()
    serie_sem_na = serie.dropna()
    dtype = str(serie.dtype).lower()

    if "age" in nome:
        return "Numérico (inteiro)"
    if "cgpa" in nome:
        return "Numérico (decimal)"
    if "current year" in nome:
        return "Categórico"

    if pd.api.types.is_numeric_dtype(serie):
        if "int" in dtype:
            return "Numérico (inteiro)"
        else:
            return "Numérico (decimal)"

    if set(serie_sem_na.unique()).issubset({"Yes","No","yes","no","Sim","Não","True","False",True,False}):
        return "Categórico (binário)"

    if serie_sem_na.nunique() < 15 and not pd.api.types.is_numeric_dtype(serie):
        return "Categórico"

    return "Textual"

def descricao_automatica(nome_coluna):
    nome = nome_coluna.lower()
    if "age" in nome:
        return "Idade do estudante"
    elif "gender" in nome or "sex" in nome:
        return "Gênero do estudante"
    elif "course" in nome or "major" in nome:
        return "Curso ou área de estudo do estudante"
    elif "depression" in nome:
        return "Presença de sintomas depressivos"
    elif "anxiety" in nome:
        return "Presença de sintomas de ansiedade"
    elif "panic" in nome:
        return "Histórico de crises de pânico"
    elif "stress" in nome:
        return "Nível de estresse do estudante"
    elif "mental" in nome:
        return "Estado geral da saúde mental do estudante"
    elif "sleep" in nome:
        return "Qualidade do sono do estudante"
    elif "exercise" in nome:
        return "Frequência de atividade física"
    elif "cgpa" in nome:
        return "Média de notas do estudante"
    elif "current year" in nome:
        return "Ano atual de estudo"
    elif "status" in nome:
        return "Status civil atual do estudante"
    elif "treatment" in nome:
        return "Se procurou atendimento médico"
    else:
        return "(Preencher manualmente)"

tabela = pd.DataFrame({
    "Nome da variável": df.columns,
    "Tipo de dado": [tipo_simplificado(df[col], col) for col in df.columns],
    "Descrição": [descricao_automatica(col) for col in df.columns],
    "Exemplos de valores": [exemplos(df[col]) for col in df.columns],
    "Dados faltantes (%)": [round(df[col].isnull().mean() * 100, 2) for col in df.columns]
})

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.width', 2000)

display(tabela)

print("\n Quantidades:")
print(f"Amostras (linhas): {df.shape[0]}")
print(f"Atributos (colunas): {df.shape[1]}")


,Nome da variável,Tipo de dado,Descrição,Exemplos de valores,Dados faltantes (%)
0,Choose your gender,Categórico,Gênero do estudante,"Female, Male",0.00
1,Age,Numérico (inteiro),Idade do estudante,"18, 21, 19, 22, 23",0.99
2,What is your course?,Textual,Curso ou área de estudo do estudante,"Engineering, Islamic education, BIT, Laws, Mathemathics",0.00
3,Your current year of Study,Categórico,Ano atual de estudo,"year 1, year 2, Year 1, year 3, year 4",0.00
4,What is your CGPA?,Numérico (decimal),Média de notas do estudante,"3.00 - 3.49, 3.50 - 4.00, 3.50 - 4.00 , 2.50 - 2.99, 2.00 - 2.49",0.00
5,Marital status,Categórico (binário),Status civil atual do estudante,"No, Yes",0.00
6,Do you have Depression?,Categórico (binário),Presença de sintomas depressivos,"Yes, No",0.00
7,Do you have Anxiety?,Categórico (binário),Presença de sintomas de ansiedade,"No, Yes",0.00
8,Do you have Panic attack?,Categórico (binário),Histórico de crises de pânico,"Yes, No",0.00
9,Did you seek any specialist for a treatment?,Categórico (binário),Se procurou atendimento médico,"No, Yes",0.00



 Quantidades:
Amostras (linhas): 101
Atributos (colunas): 10


Estilização html para PDF

In [ ]:

tabela_html = tabela.to_html(index=False, escape=False)
html_completo = f"""
<html>
<head>
<style>
    body {{ font-family: Arial, sans-serif; }}
    h2 {{ text-align: center; }}
    table {{
        width: 100%;
        border-collapse: collapse;
        font-size: 10pt;
    }}
    th, td {{
        border: 1px solid black;
        padding: 8px;
        text-align: left;
        vertical-align: top;
    }}
    th {{
        background-color: #4CAF50;
        color: white;
    }}
    tr:nth-child(even) {{background-color: #f2f2f2;}}
</style>
</head>
<body>
<h2>Tabela de Metadados - Student Mental Health</h2>
{tabela_html}
</body>
</html>
"""

In [ ]:
#Salvar
os.makedirs("/content/drive/MyDrive/CD/references", exist_ok=True)
html_path = "/content/drive/MyDrive/CD/references/tabela_metadados.html"
with open(html_path, "w", encoding="utf-8") as f:
    f.write(html_completo)

pdf_path = "/content/drive/MyDrive/CD/references/tabela_metadados.pdf"

pdfkit.from_file(html_path, pdf_path)
print(f"PDF salvo em: {pdf_path}")


PDF salvo em: /content/drive/MyDrive/CD/references/tabela_metadados.pdf
